In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# pandas series s.div()  
# pandas dataframe d.pivot_table()
tips = pd.read_csv('../pydata-book-master/ch08/tips.csv')
tips.loc[:,'tip_pct']=tips.loc[:,'tip'].div(tips.loc[:,'total_bill'])
tips.pivot_table(index=['sex','smoker'],aggfunc='mean')

In [ ]:
tips.pivot_table(values=['tip_pct','size'],index=['sex','day'],columns='smoker',aggfunc='mean')

In [ ]:
tips.pivot_table(values=['tip_pct','size'],index=['sex','day'],columns='smoker',aggfunc='mean',margins=True)

In [ ]:
tips.pivot_table(values=['tip_pct'],index=['sex','smoker'],columns='day',aggfunc=len,margins=True)

In [ ]:
tips.pivot_table(values=['size'],index=['time','sex','smoker'],columns='day',aggfunc='sum',fill_value=0)

# 交叉表：crosstab
python Built-in Functions range()  
pandas General function pd.crosstab()

In [ ]:
data =pd.DataFrame({'Sample':list(range(1,11)),
                  'Gender':['Female','Male','Female','Male','Male','Male','Female','Female','Male','Female'],
                  'Handedness':['Right-handed','Left-handed','Right-handed','Right-handed','Left-handed','Right-handed','Right-handed','Left-handed','Right-handed','Right-handed']})
data

In [ ]:
pd.crosstab(index=data.loc[:,'Gender'],columns=data.loc[:,'Handedness'],margins=True)

In [ ]:
pd.crosstab(index=[tips.loc[:,'time'],tips.loc[:,'day']],
            columns=tips.loc[:,'smoker'],margins=True)

# 示例：2012联邦选举委员会数据库
pandas Input/Output pd.read_csv()

In [ ]:
fec = pd.read_csv('../pydata-book-master/ch09/P00000001-ALL.csv',low_memory=False)
fec.info()

In [ ]:
# pandas dataframe d.iloc[]
fec.iloc[123456]

In [ ]:
# pandas dataframe d.columns
fec.columns

In [ ]:
# pandas series s.unique()
unique_cands = fec.loc[:,'cand_nm'].unique()
print(unique_cands[2])
unique_cands

In [ ]:
parties ={'Bachmann, Michelle':'Republican',
            'Romney, Mitt':'Republican',
            'Obama, Barack':'Democrat',
            "Roemer, Charles E. 'Buddy' III":'Republican',
            'Pawlenty, Timothy':'Republican',
            'Johnson, Gary Earl':'Republican',
            'Paul, Ron':'Republican',
            'Santorum, Rick':'Republican',
            'Cain, Herman':'Republican',
            'Gingrich, Newt':'Republican',
            'McCotter, Thaddeus G':'Republican',
            'Huntsman, Jon':'Republican',
            'Perry, Rick':'Republican'
         }
fec.loc[:,'cand_nm'].iloc[123456:123461]

In [ ]:
# pandas series s.map()
fec.loc[:,'cand_nm'].iloc[123456:123461].map(parties)

In [ ]:
# pandas series s.value_counts()
fec.loc[:,'party']=fec.loc[:,'cand_nm'].map(parties)
fec.loc[:,'party'].value_counts()

In [ ]:
# pandas series s.gt()
(fec.loc[:,'contb_receipt_amt'].gt(0)).value_counts()

In [ ]:
#为了简化分析，限定该数据集只能由正的出资额
# pandas series s.isin()
fec = fec.loc[fec.loc[:,'contb_receipt_amt']>0]
fec_mrbo = fec.loc[fec.loc[:,'cand_nm'].isin(['Obama, Barack','Romney, Mitt'])]

## 根据职业和雇主统计赞助信息

In [ ]:
fec.loc[:,'contbr_occupation'].value_counts().iloc[:10]

In [ ]:
# pandas series s.loc[]
tes = fec.loc[:,'contbr_occupation'].value_counts()
print(tes.loc['INFORMATION REQUESTED (BEST EFFORTS)'])
print(tes.loc['C.E.O'])

In [ ]:
# python dict d.get()
# pandas dataframe d.sum()
occ_mapping = {
    'INFORMATION REQUESTED PER BEST EFFORTS':'NOT PROVIDED',
    'INFORMATION REQUESTED':'NOT PROVIDED',
    'INFORMATION REQUESTED (BEST EFFORTS)':'NOT PROVIDED',
    'C.E.O':'CEO'
}
f = lambda x: occ_mapping.get(x,x)
fec.loc[:,'contbr_occupation']=fec.loc[:,'contbr_occupation'].map(f)

emp_mapping = {
    'INFORMATION REQUESTED PER BEST EFFORTS':'NOT PROVIDED',
    'INFORMATION REQUESTED':'NOT PROVIDED',
    'SELF':'SELF-EMPLOYED',
    'SELF EMPLOYED':'SELF-EMPLOYED',
}

f = lambda x:emp_mapping.get(x,x)
fec.loc[:,'contbr_employer'] = fec.loc[:,'contbr_employer'].map(f)

by_occupation = fec.pivot_table(values='contb_receipt_amt',
                                index='contbr_occupation',
                                columns='party',aggfunc='sum')
over_2mm = by_occupation.loc[by_occupation.sum(axis=1).gt(2000000)]
over_2mm

In [ ]:
# pandas dataframe d.plot.barh()
# 对各党派总出资额最高的职业
over_2mm.plot.barh()

In [ ]:
# 获取对Obama和Romney总出资额最高的职业和企业
# pandas GroupBy g.sum()
# pandas series s.sort_values()
# pandas series s.head()
def get_top_amounts(group,key,n=5):
    totals = group.groupby(key)['contb_receipt_amt'].sum()
    return totals.sort_values(ascending=False).head(n)

grouped = fec_mrbo.groupby('cand_nm')
grouped.apply(get_top_amounts,'contbr_occupation',n=7)

In [ ]:
grouped.apply(get_top_amounts,'contbr_employer',n=10)

## 对出资额分组

In [ ]:
bins = np.array([0,1,10,100,1000,10000,100000,1000000,10000000])
labels = pd.cut(fec_mrbo.loc[:,'contb_receipt_amt'],bins)
labels

In [ ]:
# pandas GroupBy g.size()
# pandas series s.unstack()
grouped = fec_mrbo.groupby(['cand_nm',labels])
grouped.size().unstack(level=0)

In [ ]:
bucket_sums = grouped['contb_receipt_amt'].sum().unstack(level=0)
bucket_sums

In [ ]:
#pandas dataframe d.div()
normed_sums = bucket_sums.div(bucket_sums.sum(axis=1),axis=0)
normed_sums

In [ ]:
normed_sums.iloc[:-2].plot.barh(stacked=True)

## 根据州统计赞助信息
pandas dataframe d.fillna()  
pandas dataframe d.head()

In [ ]:
grouped = fec_mrbo.groupby(['cand_nm','contbr_st'])
totals = grouped['contb_receipt_amt'].sum().unstack(level=0).fillna(0)
totals.head(10)

In [ ]:
percent = totals.div(totals.sum(1),axis=0)
percent.head(10)